In [ ]:
from ratdata import data_manager as dm, process, ingest, plot
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import scipy.signal as signal

In [ ]:
dm.db_connect('rat_data.db')

In [ ]:
beta_low = 11
beta_high = 30

In [ ]:
exceptions = {
    '2021-11-01T08-33-04 rat6 baseline': 2,
    '2021-05-24T09-59-53 rat2 - baseline': 1,
    '2021-06-17T10-23-53 rat3 baseline': 1
}

rat_peaks = dict()

for r in dm.RecordingFile.select().where(dm.RecordingFile.condition == 'baseline'):
    filename = Path(r.dirname) / r.filename
    rec = ingest.read_mce_matlab_file(filename)
    if dm.is_recording_rejected(filename.name):
        continue

    fs = int(1/rec.dt)
    
    x = np.mean(rec.electrode_data, 0)
    if dm.is_recording_sliced(r.filename):
        slice = dm.RecordingSlice.select().join(dm.RecordingFile).where(dm.RecordingFile.filename == filename.name).get()
        start = int(slice.start * fs)
        end = int((slice.start + slice.length) * fs)
        x = x[start:end]
    rat_label = dm.RecordingFile.select().where(dm.RecordingFile.filename == filename.name).get().rat.full_label

    f, psd = signal.welch(x, fs, nperseg=fs)
    m, b = process.fit_oof(f, psd, 2, 100)
    f[0] = 1e-20
    oof = f ** m * (np.e ** b)
    detrended = psd - oof
    idx = np.logical_and(f >= 10, f <= 35)
    peaks = process.find_peaks(f, detrended, beta_low, beta_high)
    if filename.stem in exceptions:
        selected_peak = exceptions[filename.stem]
    else:
        selected_peak = 0
    if rat_label in rat_peaks:
        rat_peaks[rat_label].append(peaks[selected_peak])
    else:
        rat_peaks[rat_label] = [peaks[selected_peak]]
    plt.figure(figsize=(12, 6))
    plt.plot(f[idx], detrended[idx])
    plt.title(filename.stem)
    if len(peaks) == 0:
        print('No peaks: %s' % rec.filename)
    else:
        px, py, _ = peaks[selected_peak]
        plt.plot(px, py, 'x')
    plt.savefig('plots/peaks/baseline_%s_%s.png' % (rat_label, filename.stem), facecolor='white', bbox_inches='tight')
    plt.close()

In [ ]:
plot.plot_peak_location_and_height(rat_peaks, ' (baseline recordings)', 'plots/peaks/0peak_locations_nostim.png', x_lim=[10, 30], y_lim=[-5e-5, 1.5e-4])


In [ ]:
rat_peaks_no_stim = dict()
exceptions_no_stim = {
    # '2021-05-24T10-28-49 rat2 - ST': 1
}

for r in dm.RecordingFile.select().join(dm.StimSettings).where((dm.RecordingFile.condition != 'baseline') & (dm.StimSettings.stim_type == 'nostim')):
    filename = Path(r.dirname) / r.filename
    rec = ingest.read_mce_matlab_file(filename)
    if dm.is_recording_rejected(filename.name):
        continue

    fs = int(1/rec.dt)
    
    x = np.mean(rec.electrode_data, 0)
    if dm.is_recording_sliced:
        slice = dm.RecordingSlice.select().join(dm.RecordingFile).where(dm.RecordingFile.filename == filename.name).get()
        start = int(slice.start * fs)
        end = int((slice.start + slice.length) * fs)
        x = x[start:end]

    f, psd = signal.welch(x, fs, nperseg=fs)
    m, b = process.fit_oof(f, psd, 2, 100)
    f[0] = 1e-20
    oof = f ** m * (np.e ** b)
    detrended = psd - oof
    idx = np.logical_and(f >= 10, f <= 35)
    peaks = process.find_peaks(f, detrended, beta_low, beta_high)
    if filename.stem in exceptions_no_stim:
        selected_peak = exceptions_no_stim[filename.stem]
    else:
        selected_peak = 0
    if rec.rat_label in rat_peaks_no_stim:
        rat_peaks_no_stim[rec.rat_label].append(peaks[selected_peak])
    else:
        rat_peaks_no_stim[rec.rat_label] = [peaks[selected_peak]]
    plt.figure(figsize=(12, 6))
    plt.plot(f[idx], detrended[idx])
    plt.title(filename.stem)
    if len(peaks) == 0:
        print('No peaks: %s' % rec.filename)
    else:
        px, py, _ = peaks[selected_peak]
        plt.plot(px, py, 'x')
    plt.savefig('plots/peaks/nostim_%s_%s.png' % (rec.rat_label, filename.stem), facecolor='white', bbox_inches='tight')
    plt.close()

In [ ]:
plot.plot_peak_location_and_height(rat_peaks_no_stim, ' (no stim recordings)', 'plots/peaks/0peak_locations_nostim.png', x_lim=[10, 30], y_lim=[-5e-5, 1.5e-4])

In [ ]:
rat_peaks_stim = dict()

for r in dm.RecordingFile.select().join(dm.StimSettings).where((dm.RecordingFile.condition != 'baseline') & (dm.StimSettings.stim_type != 'nostim')):
    filename = Path(r.dirname) / r.filename
    rec = ingest.read_mce_matlab_file(filename)
    if dm.is_recording_rejected(filename.name):
        continue

    fs = int(1/rec.dt)
    
    x = np.mean(rec.electrode_data, 0)
    if dm.is_recording_sliced:
        slice = dm.RecordingSlice.select().join(dm.RecordingFile).where(dm.RecordingFile.filename == filename.name).get()
        start = int(slice.start * fs)
        end = int((slice.start + slice.length) * fs)
        x = x[start:end]

    f, psd = signal.welch(x, fs, nperseg=fs)
    m, b = process.fit_oof(f, psd, 2, 100)
    f[0] = 1e-20
    oof = f ** m * (np.e ** b)
    detrended = psd - oof
    idx = np.logical_and(f >= 10, f <= 35)
    peaks = process.find_peaks(f, detrended, beta_low, beta_high)
    selected_peak = 0
    if rec.rat_label in rat_peaks_stim:
        rat_peaks_stim[rec.rat_label].append(peaks[selected_peak])
    else:
        rat_peaks_stim[rec.rat_label] = [peaks[selected_peak]]
    plt.figure(figsize=(12, 6))
    plt.plot(f[idx], detrended[idx])
    plt.title(filename.stem)
    if len(peaks) == 0:
        print('No peaks: %s' % rec.filename)
    else:
        px, py, _ = peaks[selected_peak]
        plt.plot(px, py, 'x')
    plt.savefig('plots/peaks/stim_%s_%s.png' % (rec.rat_label, filename.stem), facecolor='white', bbox_inches='tight')
    plt.close()

In [ ]:
plot.plot_peak_location_and_height(rat_peaks_stim, ' (stim recordings)', 'plots/peaks/0peak_locations_stim.png', x_lim=[10, 30], y_lim=[-1e-3, 2e-2])